<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" width="300" align="left"/>

------
# **Operando com Python e MetaTrader5**
------

**Este projeto desenvolve um sistema automatizado para análise e operação no mercado de ações, focado no Índice Bovespa (IBOV). O sistema é projetado para monitorar múltiplos ativos vinculados ao IBOV, analisando suas médias móveis e executando operações de compra e venda com base em regras predefinidas. A integração com o MetaTrader 5 permite a execução automática das operações identificadas pelo sistema.**

**Funcionalidades**
- Análise de Médias Móveis: O sistema calcula as médias móveis de diversos ativos do IBOV para identificar tendências e padrões de mercado.

- Avaliação de Condições: Verifica se os ativos atendem às condições ideais para operação com base em um conjunto de regras predefinidas, como cruzamentos de médias móveis e outras métricas técnicas.

- Execução Automática de Operações: Utiliza o MetaTrader 5 para abrir e gerenciar operações de compra e venda automaticamente, de acordo com as condições determinadas.

**Como Funciona**
- Coleta de Dados: O sistema coleta dados históricos e em tempo real dos ativos vinculados ao IBOV através de APIs financeiras ou feeds de dados.

- Cálculo de Médias Móveis: Processa os dados para calcular as médias móveis de curto e longo prazo, entre outras métricas necessárias.

- Aplicação de Regras de Operação: Com base nas regras de operação definidas, o sistema avalia quais ativos estão em conformidade e, portanto, são candidatos para operações.

- Integração com MetaTrader 5: Através da API do MetaTrader 5, o sistema envia comandos para abrir, gerenciar e fechar ordens de forma automática, facilitando uma gestão ágil e eficiente das operações.

#### *Disclaimer:*
Este código foi elaborado para fins demostrativos. O codigo aqui apresentado visa fornecer informações para auxiliar o investidor na tomada de suas próprias decisões de investimento. Reforçando, nenhum tópico aqui abordado constitui qualquer tipo de indicação/oferta/solicitação de compra/venda de qualquer produto.

# 1. Bibliotecas

In [ ]:
%pip install MetaTrader5

In [ ]:
# Análise e manipulação de dados

import numpy as np
import pandas as pd

# Dados séries temporais

from datetime import datetime, timedelta
import pytz
from time import sleep

# Bibliotecas gráficas

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Integração com MetaTrader 5

import MetaTrader5 as mt5

# 2. Iniciar o MT5 e configurar os ativos desejados

In [ ]:
mt5.initialize()

In [ ]:
lista_tickers = ["ABEV3", "ALPA4", "AZZA3", "ASAI3", "AZUL4", "B3SA3", "BBAS3",
                "BBDC3", "BBDC4", "BBSE3", "BEEF3", "BPAC11", "BPAN4", "BRAP4", "BRFS3",
                "BRKM5", "CASH3", "CCRO3", "CIEL3", "CMIG4", "CMIN3", "COGN3", "CPFE3",
                "CPLE6", "CRFB3", "CSAN3", "CSNA3", "CVCB3", "CYRE3",  "DXCO3", "ECOR3",
                "EGIE3", "ELET3", "ELET6", "EMBR3", "ENEV3", "ENGI11", "EQTL3",
                "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "HAPV3", "HYPE3", "IGTI11",
                "ITSA4", "ITUB4", "JBSS3", "KLBN11", "LREN3", "LWSA3", "MGLU3", "MRFG3",
                "MRVE3", "MULT3", "NTCO3", "PCAR3", "PETR3", "PETR4", "PETZ3", "PRIO3",
                "QUAL3", "RADL3", "RAIL3", "RAIZ4", "RDOR3", "RENT3", "RRRP3", "SANB11",
                "SBSP3", "SLCE3", "SMTO3", "SUZB3", "TAEE11", "TIMS3", "TOTS3",
                "UGPA3", "USIM5", "VALE3", "VBBR3", "VIVT3", "WEGE3", "YDUQ3",]

In [ ]:
for ticker in lista_tickers:
    selected=mt5.symbol_select(ticker,True)
    if selected:
        pass
    if not selected:
        print(ticker,"não localizado")

# 3. Criar screening de ativos

### 3.1. Organizar os dados de cotações

In [ ]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=500)
data_fim = datetime.now()
data_inicio

In [ ]:
cotacoes_mt5_all = []

for ticker in lista_tickers:
    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)

In [ ]:
list(enumerate(lista_tickers))

### 3.2. Adicionar indicadores e regras do screening

In [ ]:
for i in range(0,len(cotacoes_mt5_all)):
  cotacoes_mt5_all[i]['SMA_20p'] = cotacoes_mt5_all[i]['close'].rolling(20).mean()
  cotacoes_mt5_all[i]['SMA_200p'] = cotacoes_mt5_all[i]['close'].rolling(200).mean()

Vamos imaginar duas regra simples:

1) Ativos cuja **MM_20p seja maior que a MM_200p**
2) Ativos cujo **fechamento esteja acima da MM_20p**

In [ ]:
ls_screening_Regra_SMA = []

for i in range(0,len(cotacoes_mt5_all)):
    if (cotacoes_mt5_all[i].iloc[-1]['SMA_20p'] > cotacoes_mt5_all[i].iloc[-1]['SMA_200p']) & \
      (cotacoes_mt5_all[i].iloc[-1]['close'] > cotacoes_mt5_all[i].iloc[-1]['SMA_20p']):
        ls_screening_Regra_SMA.append((lista_tickers[i]))
        print("O ativo indexado como",i,"("+lista_tickers[i]+") atende às regras")
    else:
        # print(lista_tickers[i],"não atende")
        pass

### 3.3. Checando com a visualização gráfica

In [ ]:
n_ativo = 9

cotacoes_df = cotacoes_mt5_all[n_ativo].iloc[-30:]

fig_cotacoes = go.Figure(data=[go.Candlestick(name=cotacoes_df.ticker.iloc[n_ativo],x=cotacoes_df.index, open=cotacoes_df['open'], high = cotacoes_df['high'],
                                                low=cotacoes_df['low'], close=cotacoes_df['close'])])

fig_cotacoes.add_trace(go.Scatter(name='SMA_20p', x=cotacoes_df.index, y=cotacoes_df['SMA_20p'],marker_color='blue'))
fig_cotacoes.add_trace(go.Scatter(name='SMA_200p', x=cotacoes_df.index, y=cotacoes_df['SMA_200p'],marker_color='black'))

fig_cotacoes.update_yaxes(title_text="<b> Preço Ativo (R$)")
fig_cotacoes.update_layout(xaxis_rangeslider_visible=False, title_text='Grafico Candlestick Ativo',template = 'simple_white',width=500,height=500,showlegend=True)
fig_cotacoes.show()

# 4. Automatizando o trade baseado nas regras

**Caracteristicas da operação:**
- O preço do ativo precisa estar acima da Media movel de 20 e da média movel de 200
- Lote de 100 Ações
- Stop loss de 8 pontos
- Take profit de 8 Pontos
- Caso o ativo já esteja comprado o robô não entra em operação

In [ ]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=3)
data_fim = datetime.now()

contagem = 0

while contagem < 15:

    print('------- Inicio do',contagem,' round -------')
    for ticker in lista_tickers:

        cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, data_inicio, data_fim)
        cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
        cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
        cotacoes_mt5_ticker['ticker'] = str(ticker)
        cotacoes_mt5_ticker['SMA_rapida'] = cotacoes_mt5_ticker['close'].rolling(10).mean()
        cotacoes_mt5_ticker['SMA_lenta'] = cotacoes_mt5_ticker['close'].rolling(20).mean()

        if (cotacoes_mt5_ticker.iloc[-1]['SMA_rapida'] > cotacoes_mt5_ticker.iloc[-1]['SMA_lenta']) & \
            (cotacoes_mt5_ticker.iloc[-1]['close'] > cotacoes_mt5_ticker.iloc[-1]['SMA_rapida']):
            print("O ativo indexado como",ticker, "atende às regras e será comprado")

            positions = mt5.positions_get(symbol=ticker)
            if len(positions)==0:
                qtd = 100.0
                tick_min = mt5.symbol_info(ticker).point
                preco = mt5.symbol_info_tick(ticker).ask
                ordem_compra = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": ticker,
                    "volume": qtd,
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": preco,
                    "sl": preco - 8 * tick_min,
                    "tp": preco + 8 * tick_min,
                    "magic": 1,
                    "comment": "Automatizando trades",
                    "type_time": mt5.ORDER_TIME_DAY,
                    "type_filling": mt5.ORDER_FILLING_RETURN,
                    }
                result_compra = mt5.order_send(ordem_compra)
            else:
                print(ticker,"já posicionado")

        else:
            # print(ticker,"não atende ao screening")
            pass

        sleep(1)

    contagem += 1
    sleep(30)